In [1]:
import tarfile
import ast
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import math
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
tar = tarfile.open(".\MovieSummaries.tar.gz", encoding='utf-8')
tar.extractall()
tar.close()

In [ ]:
f = open("./MovieSummaries/movie.metadata.tsv", 'r', encoding="utf8")
data = f.readlines()
f.close()

In [ ]:
for i in range(len(data)):
    data[i] = data[i].split('\t')

In [ ]:
movie_metadata = {}
for i in range(len(data)):
    movie_metadata[data[i][0]] = {
        'title' : data[i][2],
        'release' : data[i][3],
        'revenue' : data[i][4],
        'genres' : [genre.lower() for genre in (ast.literal_eval(data[0][8][:-1])).values()]
    }

In [ ]:
f = open("./MovieSummaries/plot_summaries.txt", 'r', encoding="utf8")
plot_data = f.readlines()
f.close()

In [ ]:
movie_plot_summaries = []
for i in range(len(plot_data)):
    movie_plot_summaries.append(plot_data[i].split('\t'))

In [ ]:
for summary in movie_plot_summaries:
    if summary[0] in movie_metadata: 
        movie_metadata[summary[0]]['summary'] = summary[1]

In [ ]:
pickle.dump(movie_metadata, open("./MovieSummaries/movie_metadata.p", "wb" ))
# movie_metadata = pickle.load(open("./MovieSummaries/movie_metadata.p", "rb" ))

In [ ]:
movie_metadata2 = {}
for k,v in movie_metadata.items():
    if 'summary' in v:
        movie_metadata2[k] = v

In [2]:
# pickle.dump(movie_metadata2, open("./MovieSummaries/movie_metadata2.p", "wb" ))
movie_metadata2 = pickle.load(open("./MovieSummaries/movie_metadata2.p", "rb" ))

In [ ]:
tfidf_vec = TfidfVectorizer(min_df=5, max_df=0.95, stop_words='english')
doc_by_vocab = tfidf_vec.fit_transform([v['summary'] if 'summary' in v else '' for k,v in movie_metadata2.items()]).toarray()

In [ ]:
def build_movie_sims_jac(metadata):
    n = len(metadata)
    mat = np.zeros((n, n))
    metadata_list = [set((v['summary']).split(' ')) for k, v in metadata.items()]
    for i in range(n):
        for j in range(i+1):
            if i == j:
                mat[i][j] = 1
            else:
                jaccard_sim = len(metadata_list[i].intersection(metadata_list[j]))/len(metadata_list[i].union(metadata_list[j]))
                mat[i][j] = jaccard_sim
                mat[j][i] = jaccard_sim
    return mat

movie_sims_jac = build_movie_sims_jac(movie_metadata2) 